In [13]:
from datasets import load_dataset
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline, set_seed
import warnings
warnings.filterwarnings(action='ignore')

## CNN/DailyMail 데이터셋

In [14]:
dataset = load_dataset("cnn_dailymail", version="3.0.0") 
print(f"특성: {dataset['train'].column_names}")

특성: ['article', 'highlights', 'id']


In [15]:
sample = dataset["train"][1]
print(f"""기사 (500개 문자 발췌, 총 길이: {len(sample["article"])}):""")
print(sample["article"][:500])
print(f'\n요약 (길이: {len(sample["highlights"])}):')
print(sample["highlights"])

기사 (500개 문자 발췌, 총 길이: 4051):
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

요약 (길이: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/pjw/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
sample_text = dataset["train"][1]["article"][:2000]

In [18]:
summaries = {} # 각 모델이 생성한 요약 저장

### 요약 baseline
- 기사를 요약하는 일반적인 기준은 맨 처음 3문장을 선택하는 것

In [19]:
def sentences_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [20]:
summaries["baseline"] = sentences_summary(sample_text)

### GPT-2

In [21]:
set_seed(42)

pipe = pipeline("text-generation", model="gpt2-large")

gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### T5

In [22]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### BERT

In [23]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### 결과 비교

In [26]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."

GPT2
Solitary confinement causes a disproportionate number of serious mental illness cases in D.C.
There are a few places in the system that have mental health or behavioral treatment programs, but most jail programs have n

### 결과 평가

In [27]:
from datasets import load_metric
import pandas as pd
import numpy as np

Rouge

In [29]:
rouge_metric = load_metric("rouge")

In [30]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
    records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.316832,0.080808,0.158416,0.257426
t5,0.382979,0.130435,0.255319,0.382979
bart,0.475248,0.222222,0.316832,0.415842
